# Radix-Sort
## The program contains parallel implementation of Radix-sort algorithm which sorts an array of non-negative integers in approximately O(klogn) time where k is the number of bits in the binary representation of largest number in array and n is the number of elements in array 


In [1]:
using Pkg
Pkg.add("CuArrays")
Pkg.add("CUDAnative")
Pkg.add("CUDAdrv")
Pkg.add("StaticArrays")
Pkg.add("BenchmarkTools")
Pkg.add("Test")

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
  [be33ccc6] + CUDAnative v2.2.1
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
  [c5f51814] + CUDAdrv v3.0.1
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
  [90137ffa] + StaticArrays v0.11.0
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]
 Resolving package versions...
   Cloning [6e4b80f9-dd63-53aa-95a3-0cdb28fa8baf] BenchmarkTools from https://github.com/JuliaCI/BenchmarkTools.jl.git
  Updat

┌ Info: Installed BenchmarkTools ─ v0.4.2
└ @ Pkg.Operations /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.0/Pkg/src/Operations.jl:604


  Updating `~/.julia/environments/v1.0/Project.toml`
  [8dfed614] + Test 
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]


In [0]:
using CUDAnative, CUDAdrv, CuArrays, StaticArrays, BenchmarkTools, Test

In [3]:
# Kernel to find maximum element in array

function find_max(d_input)
  idx = (blockIdx().x - 1) * blockDim().x + threadIdx().x
  
  if(idx <= size(d_input,1))
    step = 1
    while step < blockDim().x
      if(idx % (step*2) ==1 && idx + step <= size(d_input,1))
        @inbounds d_input[idx] = max(d_input[idx],d_input[idx + step])
      end
      step *= 2
      sync_threads
    end
    if(threadIdx()==1 && idx!=1)
      @inbounds @atomic d_input[1] = max(d_input[1],d_input[idx])
    end
  end
  
  return
end

find_max (generic function with 1 method)

In [4]:
input1 = rand(1:1000,2^9)
d_input1 = CuArray(input1)
input2 = input1
d_input2 = CuArray(input2)
const N = size(input1,1)

512

In [5]:
@cuda threads = N find_max(d_input1)
d_max = Array(d_input1)[1]
d_temp1 = CuArray(zeros(Int,N))
d_temp2 = CuArray(zeros(Int,N))
d_ans = CuArray(zeros(Int,N))

512-element CuArray{Int64,1}:
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 ⋮
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0

In [6]:
# Radix sort kernel

function radix_sort(d_input2,d_max,d_temp1,d_temp2,d_ans)
  idx = (blockIdx().x - 1) * blockDim().x + threadIdx().x 
  
  # Bounds check
  if(idx <= N)
    step = 1
    while step <= d_max
      
      # Classification in terms of the bit at nth root of step position from right
      # (n varies from LSB to MSB in the binary representation of the number)
      
      if(d_input2[idx] & step == step)
        d_temp2[idx] = 1
        d_temp1[idx] = 0
      else
        d_temp1[idx] = 1
        d_temp2[idx] = 0
      end
      
      flag = 0 
      if(d_temp1[idx] == 1)
        flag = 1
      end
      sync_threads()
      
      #### Scan on d_temp1 and d_temp2 ####
      Step = 1
      while Step < N
        temp1 = d_temp1[idx]
        temp2 = d_temp2[idx]
        
        if(idx + Step <= N)
          temp1 += d_temp1[idx + Step]
          temp2 += d_temp2[idx + Step]
          d_temp1[idx + Step] = temp1
          d_temp2[idx + Step] = temp2
        end
                
        sync_threads()
        Step *= 2
      end
      ######################################
      
      # Scattering to the address obtained by scan
      if(flag == 1)
        d_ans[d_temp1[idx]] = d_input2[idx]
          @cuprintf("%ld ",d_temp1[idx])
      else
        d_ans[d_temp2[idx] + d_temp1[N]] = d_input2[idx]
          @cuprintf("%ld ",d_temp2[idx] + d_temp1[N])
      end
      d_input2[idx] = d_ans[idx]      
      step *= 2
      sync_threads()
    end
  end
  return
end

radix_sort (generic function with 1 method)

In [0]:
@cuda threads = N radix_sort(d_input2,d_max,d_temp1,d_temp2,d_ans)

In [8]:
@test isapprox(Array(d_input2),sort(input1)) 
# Test passes

364 365 366 367 368 369 370 371 372 373 374 375 389 390 391 392 393 394 395 396 397 398 399 400 401 402 403 404 274 275 276 277 278 279 280 281 282 283 284 285 286 287 466 467 468 469 470 471 472 473 474 475 476 477 478 479 480 481 482 405 406 407 408 409 410 411 412 413 414 415 416 417 418 419 420 421 422 423 424 425 426 427 428 429 430 431 432 433 434 288 289 290 291 292 293 294 295 296 297 298 299 300 483 484 485 486 487 488 489 490 491 492 493 494 495 352 353 354 355 356 357 358 359 360 361 362 363 496 497 498 499 500 501 502 503 504 505 506 507 508 509 510 511 512 435 436 437 438 439 440 441 442 443 444 445 446 447 448 449 450 376 377 378 379 380 381 382 383 384 385 386 387 388 451 452 453 454 455 456 457 458 459 460 461 462 463 464 465 301 302 303 304 305 306 307 308 309 310 311 312 313 314 315 316 335 336 337 338 339 340 341 342 343 344 345 346 347 348 349 350 351 317 318 319 320 321 322 323 324 325 326 327 328 329 330 331 332 333 334 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33

Test Passed

In [9]:
@btime @cuda threads = N radix_sort(d_input2,d_max,d_temp1,d_temp2,d_ans)

  15.138 μs (72 allocations: 2.38 KiB)


In [10]:
@benchmark @cuda threads = N radix_sort(d_input2,d_max,d_temp1,d_temp2,d_ans)

BenchmarkTools.Trial: 
  memory estimate:  2.38 KiB
  allocs estimate:  72
  --------------
  minimum time:     15.165 μs (0.00% GC)
  median time:      8.412 ms (0.00% GC)
  mean time:        7.677 ms (0.00% GC)
  maximum time:     9.154 ms (0.00% GC)
  --------------
  samples:          651
  evals/sample:     1

In [11]:
function seq_merge(left,right,input)
  i = 1
  j = 1
  k = 1
  while(i<=size(left,1) && j<= size(right,1))
    if(left[i] <= right[j])
      input[k] = left[i]
      i += 1
    else
      input[k] = right[j]
      j += 1
    end
    k += 1
  end
  while(i<=size(left,1))
    input[k] = left[i]
    k += 1
    i += 1
  end
  while(j<=size(right,1))
    input[k] = right[j]
    k += 1
    j += 1
  end
end

function seq_mergesort(input)
  if(size(input,1) < 2)
    return
  end
  mid = div(size(input,1),2)
  left = zeros(Int,mid)
  right = zeros(Int,size(input,1)-mid)
  for i=1:mid
    left[i] = input[i]
  end
  for i=mid+1:size(input,1)
    right[i-mid] = input[i]
  end
  seq_mergesort(left)
  seq_mergesort(right)
  seq_merge(left,right,input)
end

seq_mergesort (generic function with 1 method)

In [12]:
@btime seq_mergesort(input1)

  50.706 μs (1022 allocations: 121.44 KiB)


In [13]:
@benchmark seq_mergesort(input1)

BenchmarkTools.Trial: 
  memory estimate:  121.44 KiB
  allocs estimate:  1022
  --------------
  minimum time:     53.271 μs (0.00% GC)
  median time:      65.669 μs (0.00% GC)
  mean time:        92.720 μs (23.61% GC)
  maximum time:     96.087 ms (99.91% GC)
  --------------
  samples:          10000
  evals/sample:     1

## Here the time required to launch the kernel and threads has to be considered separately 
## Furthur optimizations can make code much faster